In [ ]:
path_data_2 = "/Users/vothanhnhan/Desktop/gift/AIC2/model/data2.pkl"
path_data_3 = "/Users/vothanhnhan/Desktop/gift/AIC2/model/data3.pkl"
import numpy as np
import torch
import clip
import os
from PIL import Image
import pickle
import faiss
from transformers import BertTokenizer, BertModel
from sentence_transformers import SentenceTransformer
from translates1 import *
from sequence_retrieval import *


In [ ]:
tokenizer = BertTokenizer.from_pretrained('vinai/phobert-base')
model_sen = BertModel.from_pretrained('vinai/phobert-base').to("cpu")



In [ ]:
data2 = pickle.load(open(path_data_2,'rb'))
data3 = pickle.load(open(path_data_3,'rb'))
d = 768
print(data2[:6])

In [ ]:
index = faiss.IndexFlatL2(d)
print(data2_1.shape)

In [ ]:

print(data2[:8])

In [ ]:
import numpy as np
index = faiss.IndexFlatL2(d)
def add_noise_to_vector(vector, noise_level=0.001):
    noise = np.random.normal(0, noise_level, vector.shape)
    return vector + noise

noisy_vector = add_noise_to_vector(data2)
noisy_vector /= np.linalg.norm(noisy_vector,axis=1, keepdims=True)
index.add(noisy_vector)

In [ ]:
def encode_documents(documents):
    inputs = tokenizer(documents, return_tensors="pt", padding=True, truncation=True).to("cpu")
    # Rút trích đặc trưng
    with torch.no_grad():
        outputs = model_sen(**inputs)
        feature = outputs.last_hidden_state.mean(dim=1)  # Lấy vector trung bình làm đặc trưng
    return feature.cpu().numpy()
text_search = "con chó"
find_embeddings = encode_documents(text_search).reshape((1,-1))
find_embeddings /= np.linalg.norm(find_embeddings,axis=1, keepdims=True)
#print(find_embeddings)
k = 20
D, I = index.search(find_embeddings, k)
print(I)
print(D)
k = [(data3[i][0], data3[i][2], data3[i][3]) for i in I[0]]
print(k)

In [ ]:
#[[  23   65   66  278  303  360  490  491  500  524  525  754  760  840
#   858  891  908 1071 1072 1154]]

'''
[[ 7  0  5 18  6 14  4  3  2  1  9  8 19 22 11 13 17 16 21 15]]
'''

In [ ]:

import pickle
with open('/Users/vothanhnhan/Desktop/gift/AIC2/src/data.pkl', 'rb') as file:  # Mở file ở chế độ ghi nhị phân (wb)
    src = pickle.load(file) 
print(src[:10])

In [ ]:
# Đoạn text mà bạn muốn rút trích đặc trưng

import torch
from transformers import BertTokenizer, BertModel
from tqdm import tqdm

device = "cpu"

# Tải tokenizer và mô hình TinyBERT
tokenizer = BertTokenizer.from_pretrained('vinai/phobert-base')
model = BertModel.from_pretrained('vinai/phobert-base').to(device)

texts = [src[i][3] for i in range(len(src)-1)]
print(texts[1])
# Khởi tạo danh sách để lưu trữ đặc trưng
text_features = []

# Khởi tạo tqdm để theo dõi tiến trình
for text in tqdm(texts, desc="Rút trích đặc trưng của text"):
    # Tokenize đoạn text
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True).to(device)
    
    # Rút trích đặc trưng
    with torch.no_grad():
        outputs = model(**inputs)
        feature = outputs.last_hidden_state.mean(dim=1)  # Lấy vector trung bình làm đặc trưng
        text_features.append(feature)

# Chuyển list thành tensor
#text_features = torch.cat(text_features)



In [ ]:
import xml.etree.ElementTree as ET
import os
def extract_text_from_ttml(file_path):
    # Parse the TTML file
    try:
        tree = ET.parse(file_path)
        root = tree.getroot()

        # Namespace dictionary to handle XML namespaces
        namespaces = {'tt': 'http://www.w3.org/ns/ttml'}
        # Extract text from <p> elements
        texts = []
        for elem in root.findall('.//tt:p', namespaces):
            texts.append((file_path.split('/')[-1].split('.')[0],elem.attrib.get('begin'), elem.attrib.get('end'), elem.text))
        return texts
    except ET.ParseError as e:
        print(f"Error parsing the TTML file: {e}")
        return []
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        return []
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return []

# Path to your TTML file
file_path = '/Users/vothanhnhan/Desktop/gift/AIC2/datasets/subtitle'
# Extract text
extracted_texts = []

i = 0
for path in os.listdir(file_path):
    #print(os.path.join(file_path,path))
    extracted_texts += extract_text_from_ttml(os.path.join(file_path,path))


# Print extracted texts

cnt = 0
for text in extracted_texts:
    cnt += len(text[3])
print(cnt)


In [ ]:
print(extracted_texts[:5])
src = extracted_texts

In [ ]:
import pickle
with open("data2.pkl", "wb") as f:
    pickle.dump(extracted_texts,f)

In [ ]:

from tqdm import tqdm
from sentence_transformers import SentenceTransformer

# Tạo mô hình Sentence Transformer
model_name = 'all-MiniLM-L6-v2'
model = SentenceTransformer(model_name)
model.to("cpu")

# Chuyển đổi văn bản thành các vector đặc trưng


# Danh sách các văn bản


# Chuyển đổi văn bản thành các vector đặc trưng
doc_embeddings = []
for i in tqdm(range(len(src))):
    embeddings = model.encode(src[i][3], convert_to_tensor=True, device="cpu")
    doc_embeddings.append((src[i][0], src[i][1], src[i][2], src[i][3], embeddings.cpu().numpy()))


In [ ]:
print(doc_embeddings[0])

In [8]:
import pickle
with open("/Users/vothanhnhan/Desktop/gift/AIC2/model/data2_1.pkl","rb") as f:
    src = pickle.load(f)

In [12]:
text_find = "con mèo"
for src_tmp in src:
    if text_find in src_tmp[3]:
        print(src_tmp[3])

Pháp một con cá hai con mèo một con cầu
một con mèo và một con chim mưng hai xác
con mèo trên thuyền chú mèo II và người
để đặt cho những con mèo sắp bị triệt
được đặt cho con mèo hoang đã bị sắp
một con mèo cái có thể đẻ tới hai ba lứa
600 con mèo hoang mèo sau khi Triệt sản
chó ho con mèo đó vẫn có thể truyền
con mèo trên thuyền như
chó và một con mèo tham gia cuộc thi sự
nhân J biden hóa trang thành một con mèo
cây mới thu hút sự chú ý của các con mèo
bản vẻ một con mèo đang ngủ rồi Anh cùng
con mèo đang ngủ và quắp con cá Bước
hiện trong sự kiện còn có Mouse con mèo
những con mèo may mắn như thế này nó rất


In [ ]:
print(src[100])

In [ ]:
import faiss
import numpy as np
index = faiss.IndexFlatL2(768)
x = np.array([tmp[4] for tmp in src]).reshape(-1,768)
x /= np.linalg.norm(x,axis=1, keepdims=True)
index.add(np.array(x).reshape(-1,768))
def encode_documents(documents):
    inputs = tokenizer(documents, return_tensors="pt", padding=True, truncation=True).to("cpu")
    # Rút trích đặc trưng
    with torch.no_grad():
        outputs = model(**inputs)
        feature = outputs.last_hidden_state.mean(dim=1)  # Lấy vector trung bình làm đặc trưng
    return feature.cpu().numpy()
def encode(documents):
    embeddings = model.encode(documents, convert_to_tensor=True, device="cpu")
    return  embeddings.cpu().numpy()

text_search = "7:00"
find_embeddings = encode_documents(text_search).reshape((1,-1))
find_embeddings /= np.linalg.norm(find_embeddings,axis=1, keepdims=True)
#print(find_embeddings)
k = 20
D, I = index.search(find_embeddings, k)
print(I)
print(D)
tm = [doc_embeddings[i][3] for i in I[0]]
print(tm)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def similarity(vector_1, vector_2):
    """
    Tính toán độ tương đồng cosin giữa hai vector.
    
    Parameters:
    - vector_1: np.array, vector đầu tiên.
    - vector_2: np.array, vector thứ hai.
    
    Returns:
    - similarity_score: float, điểm số độ tương đồng giữa hai vector.
    """
    # Đảm bảo các vector là 2D để sử dụng cosine_similarity
    vector_1 = np.atleast_2d(vector_1)
    vector_2 = np.atleast_2d(vector_2)
    
    # Tính toán độ tương đồng cosin
    similarity_score = cosine_similarity(vector_1, vector_2)
    
    return similarity_score[0][0]

# Ví dụ sử dụng
vector_a = np.array([1, 2, 3])
x = np.array([tmp[4] for tmp in doc_embeddings]).reshape(-1,384)
x /= np.linalg.norm(x,axis=1, keepdims=True)
text_search = "hoa kỳ"
find_embeddings = encode(text_search).reshape((1,-1))

similarity_score = [(i, similarity(find_embeddings,x[i])) for i in range(x.shape[0])]
similarity_score.sort(key = lambda x: x[1],reverse=True)


In [ ]:
print(similarity_score[:10])

In [ ]:
print([doc_embeddings[similarity_score[tmp][0]][3] for tmp in range(20)])